INTRODUCTION

I am looking to find which area of Mumbai would be the best for me to open a snack place. In India, many people are below the poverty line and opening small businesses is the most popular way for citizens to provide for themselves and their families. Knowing where to open their store can help them run a better business as they are setting up shop in an area where they are more likely to succeed


I will be using wikipedia to webscrape the list of suburbs in Mumbai and merge it with location data of their longitude and latitude using geocoder. I will then use the Foursquare API to see what kind of venues are in each area to see where most of the snack shop density is. 

In [159]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


In [274]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

#https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai
#https://en.wikipedia.org/wiki/Category:Cities_in_the_San_Francisco_Bay_Area

In [275]:
soup = BeautifulSoup(data, 'html.parser')

In [276]:
# create a list to store neighborhood data
neighborhoodList = []

In [277]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [278]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [279]:
# print the number of rows of the dataframe
kl_df.shape

(40, 1)

In [280]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [281]:
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [282]:
coords

[[19.11848309908247, 72.84177419095158],
 [19.042830000000038, 72.92734000000007],
 [19.06293000000005, 72.92666000000008],
 [19.054220000000043, 72.84019000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.062200000000075, 72.90242000000006],
 [19.250030000000038, 72.85908000000006],
 [19.224720000000048, 72.86606000000006],
 [19.212870000000066, 73.08324000000005],
 [19.00538889189226, 72.85576887678867],
 [19.086476606699875, 72.9089562772808],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.13790000000006, 72.84941000000003],
 [19.01493000000005, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21195211212422, 72.83754191243007],
 [19.131400000000042, 72.93565000000007],
 [19.127560000000074, 72.82540000000006],
 [19.064940000000036, 72.88073000000003],
 [19.21195211212422, 72.83754191243007],
 [19.048530000000028, 72.93220000000008],


In [284]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(40, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118483,72.841774
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062930,72.926660
3,Bandra,19.054220,72.840190
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062200,72.902420
8,Dahisar,19.250030,72.859080
9,Devipada,19.224720,72.866060


In [285]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [286]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [287]:
# define Foursquare Credentials and Version
CLIENT_ID = '1O2BMISUJQUUSKNGXJ3A1H1MWAA3E1VO0CVXDYDNMXWMORVA' # your Foursquare ID
CLIENT_SECRET = 'YESPOKVIGYAENNYXDDELYFAHX5WDH2GIWIRZORQLZJT0CM2V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1O2BMISUJQUUSKNGXJ3A1H1MWAA3E1VO0CVXDYDNMXWMORVA
CLIENT_SECRET:YESPOKVIGYAENNYXDDELYFAHX5WDH2GIWIRZORQLZJT0CM2V


In [288]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [289]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2656, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118483,72.841774,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118483,72.841774,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118483,72.841774,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118483,72.841774,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop
4,Andheri,19.118483,72.841774,Tewari Bros Sweets,19.115305,72.834501,Indian Restaurant


In [290]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,18,18,18,18,18,18
Baiganwadi,8,8,8,8,8,8
Bandra,100,100,100,100,100,100
Bhandup,22,22,22,22,22,22
Borivali,100,100,100,100,100,100
Charkop,54,54,54,54,54,54
Chembur,95,95,95,95,95,95
Dahisar,64,64,64,64,64,64


In [291]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 195 uniques categories.


In [292]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop', 'Coffee Shop',
       'Falafel Restaurant', 'Fast Food Restaurant', 'Pub', 'Café',
       'Pizza Place', 'Juice Bar', 'Multiplex', 'Snack Place',
       'Breakfast Spot', 'Sandwich Place', 'Cocktail Bar', 'Bar',
       'American Restaurant', 'Electronics Store', 'Chinese Restaurant',
       'Seafood Restaurant', 'Department Store', 'Diner', 'Spa',
       'Vegetarian / Vegan Restaurant', 'Asian Restaurant', 'BBQ Joint',
       'Residential Building (Apartment / Condo)', 'Lounge',
       'Gym / Fitness Center', "Women's Store", 'Liquor Store',
       'Smoke Shop', 'Food Truck', 'Athletics & Sports', 'Fish Market',
       'Hotel', 'Martial Arts Dojo', 'Tea Room', 'Clothing Store', 'Food',
       'Plaza', 'Sports Bar', 'Park', 'Platform',
       'Paper / Office Supplies Store', 'Food & Drink Shop',
       'Music Venue', 'Tunnel', 'Gym', 'Dessert Shop'], dtype=object)

In [293]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [294]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2656, 196)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0

In [295]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(40, 196)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.000000,0.0000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.020000,0.000000,0.040000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.050000,0.000000,0.00,0.050000,0.010000,0.02,0.020000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.020000,0.000000,0.0000,0.010000,0.00,0.000000,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.040000,0.170000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.010000,0.020000,0.000000,0.000000,0.01,0.0000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.

In [299]:
len(kl_grouped[kl_grouped["Snack Place"] > 0])

21

In [300]:
kl_mall = kl_grouped[["Neighborhoods","Snack Place"]]

In [301]:
kl_mall.head()

,Neighborhoods,Snack Place
0,Andheri,0.05
1,Anushakti Nagar,0.00
2,Baiganwadi,0.00
3,Bandra,0.02
4,Bhandup,0.00


In [302]:
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 0, 1, 2, 1, 0, 0, 0], dtype=int32)

In [303]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [304]:

kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Snack Place,Cluster Labels
0,Andheri,0.05,2
1,Anushakti Nagar,0.00,1
2,Baiganwadi,0.00,1
3,Bandra,0.02,0
4,Bhandup,0.00,1


In [305]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(40, 5)


,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
0,Andheri,0.05,2,19.118483,72.841774
1,Anushakti Nagar,0.00,1,19.042830,72.927340
2,Baiganwadi,0.00,1,19.062930,72.926660
3,Bandra,0.02,0,19.054220,72.840190
4,Bhandup,0.00,1,19.145560,72.948560


In [306]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(40, 5)


,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
22,Mahavir Nagar (Kandivali),0.012500,0,19.211952,72.837542
14,Grant Road,0.020000,0,18.959290,72.831080
13,Goregaon,0.020000,0,19.164550,72.849460
12,Ghatkopar,0.012500,0,19.086477,72.908956
24,"Matharpacady, Mumbai",0.020408,0,19.044920,72.867205
18,Kandivali,0.012500,0,19.211952,72.837542
9,Devipada,0.022222,0,19.224720,72.866060
29,Pestom sagar,0.023810,0,19.070630,72.902200
8,Dahisar,0.015625,0,19.250030,72.859080
32,"Sion, Mumbai",0.015873,0,19.046460,72.863130


In [307]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [308]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
22,Mahavir Nagar (Kandivali),0.012500,0,19.211952,72.837542
14,Grant Road,0.020000,0,18.959290,72.831080
13,Goregaon,0.020000,0,19.164550,72.849460
12,Ghatkopar,0.012500,0,19.086477,72.908956
24,"Matharpacady, Mumbai",0.020408,0,19.044920,72.867205
18,Kandivali,0.012500,0,19.211952,72.837542
9,Devipada,0.022222,0,19.224720,72.866060
29,Pestom sagar,0.023810,0,19.070630,72.902200
8,Dahisar,0.015625,0,19.250030,72.859080
32,"Sion, Mumbai",0.015873,0,19.046460,72.863130


In [309]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
26,Mogra Village,0.0,1,26.135300,73.05484
25,Mira Road,0.0,1,19.074161,72.86167
19,Kanjurmarg,0.0,1,19.131400,72.93565
31,Shil Phata,0.0,1,19.146580,73.04005
33,Thakur village,0.0,1,19.210200,72.87541
34,Tilak Nagar (Mumbai),0.0,1,18.996160,72.85281
28,Mumbra,0.0,1,19.190560,73.02267
30,Seven Bungalows,0.0,1,19.128560,72.82085
39,Worli,0.0,1,19.007450,72.81691
21,Kurla,0.0,1,19.064940,72.88073


In [310]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
27,Mulund,0.039474,2,19.171850,72.955640
16,Juhu,0.030000,2,19.014930,72.845220
11,Eastern Suburbs (Mumbai),0.041667,2,19.005389,72.855769
5,Borivali,0.030000,2,19.229360,72.857510
37,Wadala,0.041237,2,19.017160,72.858130
0,Andheri,0.050000,2,19.118483,72.841774
